# Knapsack example

Goal: Maximize the value of items in a bag constrained by the maximum weight the bag can carry.  At most one of each item can go in the bag.

## **Step 0**: Setup

In [1]:
# General imports
import numpy as np

# Qiskit ansatz circuits
from qiskit.circuit.library import TwoLocal

# Qiskit primitives
from qiskit.primitives import Estimator as QiskitEstimator
from qiskit.primitives import Sampler as QiskitSampler

# Qiskit runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session

# quadratic_program
from quadratic_program import QuadraticProgram

# Docplex - classical description of optimization problems
from docplex.mp.model import Model

# translations
from translators import docplex_mp_to_qp
from translators import qubo_to_sparse_pauli_op

# workflows 
from workflows import QuadraticProgramPostprocess, QuadraticProgramConverter

# SPSA
from spsa import minimize_spsa

# Middleware
import os
from quantum_serverless import Provider, distribute_program, distribute_task

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


## Load IBM Quantum Middleware

In [2]:
provider = Provider(
#    token="YOUR_TOKEN_FROM_STAGING",
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000")
)
provider

<Provider: gateway-provider>

## **Step 1** Map the problem to a Quantum Native format (Set of Operators, and a set of Quantum Circuits)

Specify optimization problem using docplex and convert to Quadratic Unconstained Binary Opimization (QUBO) problem that can be cast as an Ising Hamiltonian suitable for a quantum solution.

In [3]:
value_of_items = [4, 3, 5, 6, 7]
weight_of_items = [2, 2, 4, 5, 6]
max_weight = 12

In [4]:
def build_model():
    mdl = Model(name="0-1 Knapsack problem")
    x = {i: mdl.binary_var(name=f"x_{i}") for i in range(len(value_of_items))}
    mdl.maximize(mdl.sum(value_of_items[i] * x[i] for i in x))
    mdl.add_constraint(mdl.sum(weight_of_items[i] * x[i] for i in x) <= max_weight);
    return mdl

### Convert to `QuadraticProgram` format

In [5]:
def build_quadratic_program(mdl):
    qp = docplex_mp_to_qp(mdl)
    return qp

### Classical transformation to QUBO problem and Ising Hamiltonian

In [6]:
def build_quadratic_transformer(qp):
    quadratic_transformer = QuadraticProgramConverter()
    qubo = quadratic_transformer.run(qp)
    hamiltonian, offset = qubo_to_sparse_pauli_op(qubo)
    return quadratic_transformer, qubo, hamiltonian

### Select ansatz circuit from circuit library

In [7]:
def quantum_solution_setup(hamiltonian):
    ansatz = TwoLocal(hamiltonian.num_qubits, 'ry', 'cx', 'linear', reps=1)
    return ansatz

## **Step 2**: Optimize the circuits and the operators to be measured

NONE

## **Step 3**: Execute using a quantum primitive function (estimator or sampler)


### Standard cost function definition

In [8]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

### Setup estimator and sampler instances

In [9]:
def setup_estimator_sampler():
    estimator = QiskitEstimator(options={"shots": int(1e4)})
    sampler = QiskitSampler(options={"shots": int(1e4)})
    return estimator, sampler

### Perform minimization

In [10]:
def minimize(ansatz, hamiltonian, estimator):
    x0 = 2*np.pi*np.random.random(size=ansatz.num_parameters)
    res = minimize_spsa(cost_func, x0,
                        args=(ansatz, hamiltonian, estimator),
                        maxiter=100)
    return res

### Computute distribution at found minimum

In [11]:
def compute_distribution(ansatz, sampler, res):
    # Assign solution parameters to ansatz
    qc = ansatz.assign_parameters(res.x)
    qc.measure_all()
    samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
    return samp_dist

## **Step 4**: Post-processing of the results to return either a plot or the answer
Transform quantum solution and convert back into classical variable space

In [12]:
def post_processing(qubo, quadratic_transformer, samp_dist):
    solution = QuadraticProgramPostprocess(qubo, quadratic_transformer).run(samp_dist)
    return solution

### Setup Middleware

In [13]:
@distribute_program(provider, working_dir="./", dependencies=["docplex"])
def program_knapsack():

    # Step 1
    
    mdl = build_model()
    print(mdl.export_as_lp_string())
    
    qp = build_quadratic_program(mdl)
    print(qp.prettyprint())
    
    quadratic_transformer, qubo, hamiltonian = build_quadratic_transformer(qp)
    print(qubo.prettyprint())
    print(hamiltonian)
    
    ansatz = quantum_solution_setup(hamiltonian)
    
    # Step 3 (there is no step 2)
    
    estimator, sampler = setup_estimator_sampler()
    
    res = minimize(ansatz, hamiltonian, estimator)
    print(res)
    
    samp_dist = compute_distribution(ansatz, sampler, res)
    
    solution = post_processing(qubo, quadratic_transformer, samp_dist)
    print(solution)
    
    return {
        "value": sum(np.array(value_of_items)*solution),
        "weight": sum(np.array(weight_of_items)*solution)
    }

In [14]:
job = program_knapsack()
job

<Job | 94d01d1d-9c59-44b7-b430-e1a0eeed873b>

In [16]:
job.status()

'SUCCEEDED'

### Intepretation of solution

In [17]:
import json

middleware_res = json.loads(job.result())
print('Solution value:', middleware_res["value"])
print('Solution weight:', middleware_res["weight"])

Solution value: 16.0
Solution weight: 12.0


In [18]:
print(job.logs())

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: 0-1 Knapsack problem

Maximize
 obj: 4 x_0 + 3 x_1 + 5 x_2 + 6 x_3 + 7 x_4
Subject To
 c1: 2 x_0 + 2 x_1 + 4 x_2 + 5 x_3 + 6 x_4 <= 12

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4
End

Problem name: 0-1 Knapsack problem

Maximize
  4*x_0 + 3*x_1 + 5*x_2 + 6*x_3 + 7*x_4

Subject to
  Linear constraints (1)
    2*x_0 + 2*x_1 + 4*x_2 + 5*x_3 + 6*x_4 <= 12  'c0'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4

Problem name: 0-1 Knapsack problem

Minimize
  26*c0@int_slack@0^2 + 104*c0@int_slack@0*c0@int_slack@1
  + 208*c0@int_slack@0*c0@int_slack@2 + 260*c0@int_slack@0*c0@int_slack@3
  + 104*c0@int_slack@1^2 + 416*c0@int_slack@1*c0@int_slack@2
  + 520*c0@int_slack@1*c0@int_